# SnapShot preprocessing:
In this notebook we get the MATSim snapShot output as the input. MATSim gives us the user location in constant intervals. It does not report anything when the user is not traveling. So to make this data as much as possible closer to the actual periodic location update, we have to fill in the empty records. also we initially get the snapshots every 30 seconds. For other polling intervals we only reduce the accuracy. Also for the spatial aggregation to the level os TAZ zones, we use arcGIS to extract the associated TAZ of each location record from the snapShot file. The output of arcGIS is the input here.

#### adding required packages

In [9]:
import pandas as pd
import time
from math import floor
import numpy as np
import pickle

#### specifying the saving location 

In [3]:
savingLoc = "Y:/ZahraEftekhar/phase4/"

#### preparing the output of arcGIS for completion

The output of GIS misses the locations outside of Amsterdam. Therefore, we complete the data by considering their TAZ code `0`.

In [33]:
precompletion = pd.read_csv('{a}GISoutput_PreCompletion.CSV'.format(a=savingLoc),usecols=['mzr_id', 'VEHICLE','TIME','EASTING','NORTHING'])
precompletion = precompletion.sort_values(by=["VEHICLE","TIME"])
precompletion = precompletion.reset_index(drop=True)
with open('{a}snapShot_allowedUsers.pickle'.format(a=savingLoc),'rb') as handle:
    MATSimOutput = pickle.load(handle)
MATSimOutput=MATSimOutput.reset_index(drop=False)
print(len(MATSimOutput)-len(precompletion)," records are missing that we refill them in our snapShot. ")


7481  records are missing that we refill them in our snapShot. 


In [34]:
completeData = pd.merge(precompletion, MATSimOutput, how='right', on=['VEHICLE','TIME'])
(completeData.mzr_id[completeData.mzr_id.isna()]) = 0
completeData = completeData.loc[:,['VEHICLE','TIME','EASTING_y','NORTHING_y','mzr_id']]
completeData.columns = ['VEHICLE', 'TIME', 'EASTING', 'NORTHING', 'mzr_id']
completeData = completeData.sort_values(by = ['VEHICLE', 'TIME'])
with open('{a}finalInputPython.pickle'.format(a=savingLoc),'wb') as handle:
    pickle.dump(completeData, handle, protocol=pickle.HIGHEST_PROTOCOL)

c:\programdata\anaconda3\envs\extractingodfromxml\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
